In [6]:
import json_lines
import gzip
import json

from soft404.utils import cleaned_selector, selector_to_text

with gzip.open('../text_items.jl.gz', 'wt') as outf:
    with json_lines.open('../crawler/items.jl.gz', broken=True) as f:
        for item in f:
            sel = cleaned_selector(item.pop('html'))
            text_item = {
                'url': item['url'],
                'text': selector_to_text(sel),
                'title': xpath('//title//text()').extract().join('\n'),
                'status': item['status'],
            }
            outf.write(json.dumps(text_item))
            outf.write('\n')

In [20]:
import json_lines

with json_lines.open('../crawler/items.jl.gz', broken=True) as f:
    for i, item in enumerate(f):
        if i > 100:
            break

In [21]:
from soft404.utils import _cleaned_html_tree
tree = _cleaned_html_tree(item['html'])
sel = parsel.Selector(root=tree, type='html')

In [31]:
# this is not quite what I want - I want to avoid breaking text on inline nodes
len(list(x for x in (x.strip() for x in tree.itertext()) if x))

212

In [33]:
BLOCK_TAGS = {
    'address', 'article', 'aside', 'audio', 'blockquote', 'canvas', 'dd',
    'div', 'dl', 'fieldset', 'figcaption', 'figure', 'footer', 'form',
    'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8',
    'header', 'hgroup', 'hr', 'noscript', 'ol', 'output', 'p', 'pre',
    'section', 'table', 'tfoot', 'ul', 'video',
}

def get_blocks(tree):
    text_blocks = []
    current = []
    tag = None
    for node in tree.iter():
        if node.tag in BLOCK_TAGS:
            tag = node.tag
            if current:
                text_blocks.append((tag, ' '.join(current)))
                current = []
        if node.text:
            text = node.text.strip()
            if text:
                current.append(text)
    return text_blocks

# TODO - also get parent block tag?

text_blocks = get_blocks(tree)
len(text_blocks), text_blocks

(74,
 [('aside', 'VK on the App Store'),
  ('ul', 'Open Menu Close Menu Apple Shopping Bag'),
  ('aside',
   'Apple Mac iPad iPhone Watch TV Music Support Search apple.com Shopping Bag'),
  ('div', 'Overview Music Video Charts'),
  ('div',
   "Opening the iTunes Store. If iTunes doesn't open, click the iTunes application icon in your Dock or on your Windows desktop. Progress Indicator"),
  ('div',
   "Opening the iBooks Store. If iBooks doesn't open, click the iBooks app in your Dock. Progress Indicator"),
  ('h2', 'iTunes'),
  ('p',
   "iTunes is the world's easiest way to organize and add to your digital media collection."),
  ('div',
   'We are unable to find iTunes on your computer. To download the free app VK by VKontakte, get iTunes now.'),
  ('div',
   'Already have iTunes? Click I Have iTunes to open it now. iTunes for Mac + PC'),
  ('h2', 'VK'),
  ('div', 'By VKontakte'),
  ('p', 'View More by This Developer'),
  ('div', 'Open iTunes to buy and download apps.'),
  ('p', 'Descr